# rigidregistration: Quick walk through
The code below provides a quick walk-through demonstrating use of the rigidregistration python package.

If you find this code useful in your own research, please cite the associated publication:
"Image registration of low signal-to-noise cryo-STEM data", Ultramicroscopy (2018), DOI: 10.1016/j.ultramic.2018.04.008

### Getting started
In this example, data which is formatted as .tif files are loaded using the tifffile package.  For other file formats common to electron microscopy data (e.g., .dm3, .ser...) we recommend the excellent hyperspy package for i/o handling.  See hyperspy.org.

In [1]:
# Import libraries and functions
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
import rigidregistration

%matplotlib

Using matplotlib backend: MacOSX


In [2]:
# Load data and instantiate imstack object  

f="sample_data/SrTiO3.tif"                # Filepath to data
stack=np.rollaxis(imread(f),0,3)/float(2**16)           # Rearrange axes and normalize data
s=rigidregistration.stackregistration.imstack(stack)    # Instantiage imstack object.
s.getFFTs() 

In [3]:
# Inspect data in preparation for registration

for i in range(5,10):                      # Select which images from the stack to display
    fig,(ax1,ax2)=plt.subplots(1,2)
    ax1.matshow(stack[:,:,i],cmap='gray')
    ax2.matshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(stack[:,:,i])))),cmap='gray',vmin=np.average(np.log(np.abs(np.fft.fft2(stack[:,:,i]))))) 
    ax1.grid(False)
    ax2.grid(False)
    plt.show()

### Fourier masking
A Fourier mask is used to avoid incorrect cross correlations, by weighting more trustworthy information in frequency space more heavily.

In [4]:
# Observe the effects of varying the cutoff frequency, n, to determine the best mask.

masktype="hann"

i,j = 5,9                                    # Choose image pair
for n in np.arange(6,16,2):                  # Select n values to test
    s.makeFourierMask(mask=masktype,n=n)     # Set the selected Fourier mask
    s.show_Fourier_mask(i=i,j=j)             # Display the results

In [5]:
# Choose best mask

masktype="hann"
n=8.7

s.makeFourierMask(mask=masktype,n=n)
s.show_Fourier_mask(i=i,j=j)

### Setting Gaussian fit parameters
Gaussian fits are used to identify maxima in cross correlations. The show_Gaussian_fit function <span style="color:red">(New in 0.2.1)</span> displays the results on a single image pair to allow parameters to be optimized.


In [6]:
findMaxima = 'gf'
s.setGaussianFitParams(num_peaks=5,sigma_guess=10,window_radius=10)
s.show_Gaussian_fit(13,19)

### Calculate image shifts
Calculate the relative shifts between all pairs of images from their cross correlations.

In [7]:
s.findImageShifts(findMaxima=findMaxima);     # Find shifts. 

  0%|          | 0/190 [00:00<?, ?it/s]

### Find and correct outliers in shift matrix
The previous step determines the relative shifts between all pairs of images.  Here, any incorrectly calculated shifts -- which may result from noisy, low SNR data -- are identified and corrected.  First, the shift matrix is displayed and inspected.  Next, outliers are identified.  Outliers are then corrected.

In [8]:
# Show Xij and Yij matrices

s.show_Rij(colorbars=True)                # Disable colorbars with colorbars=False

In [9]:
# Identify outliers

s.get_outliers(threshold=10)              # Set outlier threshhold


s.mask_off_diagonal(False)                # Remove far off diagonal elements
                                          # Argument is number of off diagonals to retain
                                          # set to False to keep all

s.refine_mask()                           # Quick checks to make shift matrix correctable
                                          # e.g. unmasking diagonals


s.show_Rij(colorbars=True)

In [10]:
# Correct outliers

s.make_corrected_Rij()                    # Correct outliers using the transitivity relations
s.show_Rij_c(colorbars=True)              # Display the corrected shift matrix

### Calculate average image

To obtain the average image, each image in the stack is shifted by an amount which is calculated from the shift matrix.  The entire, shifted image stack is then averaged. 

Several functions are available for displaying and saving the resulting average image, and for summarizing the processing that's been applied to the data for quick review.

In [11]:
# Create registered image stack and average

s.get_averaged_image()

  0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
# Display final image

s.show()

In [13]:
# Display report of registration procedure

s.show_report(colorbars=True)

In [14]:
# Save report of registration procedure

s.save_report("sample_data/sample_report.pdf",colorbars=True)

In [15]:
# Save the average image

s.save("sample_data/sample_output.tif")

<span style="color:red">(New in 0.2.1)</span>

In [17]:
# Save the registered stack

s.save_registered_stack("sample_data/sample_output_stack.tif")

### Apply shifts to different stack
<span style="color:red">(New in 0.2.1)</span>
The shifts identified in this registration can be applied to another stack, e.g. a simultaneous aquisition on a different detector. Here, the same stack (`stack`)is used, but to use a different one just change `stack` to the image stack of choice.

In [18]:
s2 = s.apply_shifts_to_stack(stack)
s2.get_averaged_image(get_shifts=False, correct_Rij=False)
s2.show() 

  0%|          | 0/20 [00:00<?, ?it/s]